<a href="https://colab.research.google.com/github/nvnsudharsan/era5_to_prism/blob/main/era5_to_prism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install cdsapi xarray netCDF4 rioxarray geopandas matplotlib
!pip install -q xarray zarr gcsfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.4/205.4 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 122.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 6.8 MB/s eta 0:00:00


In [3]:
import xarray as xr
ds = xr.open_zarr(
    'gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3',
    chunks=None,
    storage_options={"token": "anon"}
)
ds = ds.sel(time=slice(ds.attrs['valid_time_start'], ds.attrs['valid_time_stop']))

In [4]:
tp = ds['total_precipitation'].sel(
    time=slice("2015-01-01", "2024-12-31"),
    latitude=slice(31.0, 29.5),  # South to north!
    longitude=slice(262.0, 263.5)  # 360-based: 360 - 98 = 262
)

In [ ]:
tp_mm = tp * 1000  # Convert to mm
tp_daily = tp_mm.resample(time='1D').sum()
tp_daily.name = "tp_mm_day"

In [ ]:
tp_daily.to_netcdf("/content/drive/MyDrive/era5_tp_daily_austin_2015_2024_zarr_gcs.nc")

In [ ]:
!pip install earthengine-api geemap --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.6/462.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.1 MB/s eta 0:00:00
  Attempting uninstall: earthengine-api
    Found existing installation: earthengine-api 1.5.18
    Uninstalling earthengine-api-1.5.18:
      Successfully uninstalled earthengine-api-1.5.18


In [3]:
import ee
ee.Authenticate()  # Run this to get the code
ee.Initialize(project='ee-my-naveensudharsan')

In [ ]:
import geemap
from datetime import datetime

# Define time range
start = '2015-01-01'
end = '2024-12-31'

# Austin region
region = ee.Geometry.Rectangle([-98.0, 29.5, -96.5, 31.0])

# PRISM daily precipitation
collection = ee.ImageCollection('OREGONSTATE/PRISM/AN81d') \
    .filterDate(start, end) \
    .filterBounds(region) \
    .select('ppt')

# Convert to a multi-band image (each day as a band)
def format_band(img):
    date_str = img.date().format('YYYYMMdd')
    return img.rename(date_str)

prism_bands = collection.map(format_band)
prism_image = prism_bands.toBands()

# Export to Drive
task = ee.batch.Export.image.toDrive(
    image=prism_image.clip(region),
    description='PRISM_Precip_2015_2024_Austin',
    folder='era5_downscaled',  # GDrive folder
    fileNamePrefix='prism_ppt_austin_2015_2024',
    region=region,
    scale=800,
    maxPixels=1e13,
    fileFormat='GeoTIFF'
)
task.start()

In [ ]:
import time

while task.active():
    print('Exporting...', task.status()['state'])
    time.sleep(60)

Exporting... RUNNING
Exporting... RUNNING
Exporting... RUNNING
Exporting... RUNNING
Exporting... RUNNING


In [ ]:
import rioxarray as rxr
import xarray as xr

# Load PRISM high-res reference
prism = rxr.open_rasterio('/content/drive/MyDrive/era5_downscaled/prism_ppt_austin_2015_2024.tif', masked=True).squeeze()

# Load ERA5 daily file you saved earlier
era5 = xr.open_dataset('/content/drive/MyDrive/era5_tp_daily_austin_2015_2024_zarr_gcs.nc')['tp_mm_day']

# Match CRS and align to PRISM grid
era5_aligned = era5.rio.write_crs("EPSG:4326").rio.reproject_match(prism)

# Save aligned ERA5 to Drive
era5_aligned.to_netcdf('/content/drive/MyDrive/era5_downscaled/era5_tp_aligned_to_prism.nc')


In [ ]:
import pandas as pd

# Load ERA5 already aligned to PRISM grid
era5 = xr.open_dataset('/content/drive/MyDrive/era5_downscaled/era5_tp_aligned_to_prism.nc')['tp_mm_day']

# Load PRISM multiband GeoTIFF
prism = rxr.open_rasterio('/content/drive/MyDrive/era5_downscaled/prism_ppt_austin_2015_2024.tif', masked=True)

# Convert to DataArray with time dimension
dates = pd.date_range('2015-01-01', periods=prism.shape[0])
prism = prism.assign_coords(band=dates).rename(band='time')

In [ ]:
# Remove single 'band' dim and set coordinates correctly
prism = prism.squeeze(drop=True)
if 'spatial_ref' in prism.coords:
    prism = prism.drop_vars('spatial_ref')

In [ ]:
trainval = prism.sel(time=slice('2015-01-01', '2022-12-31'))
test = prism.sel(time=slice('2023-01-01', '2024-12-31'))

# Split 80/20 train/val
train_dates = trainval.time.to_index()
split_idx = int(0.8 * len(train_dates))
train = trainval.sel(time=train_dates[:split_idx])
val = trainval.sel(time=train_dates[split_idx:])

# Same for ERA5
era5_train = era5.sel(time=train.time)
era5_val = era5.sel(time=val.time)
era5_test = era5.sel(time=test.time)

In [ ]:
train.name = "tp_mm_day"
val.name = "tp_mm_day"
test.name = "tp_mm_day"

era5_train.name = "tp_mm_day"
era5_val.name = "tp_mm_day"
era5_test.name = "tp_mm_day"

In [ ]:
train.to_netcdf('/content/drive/MyDrive/era5_downscaled/prism_train.nc')
val.to_netcdf('/content/drive/MyDrive/era5_downscaled/prism_val.nc')
test.to_netcdf('/content/drive/MyDrive/era5_downscaled/prism_test.nc')

era5_train.to_netcdf('/content/drive/MyDrive/era5_downscaled/era5_train.nc')
era5_val.to_netcdf('/content/drive/MyDrive/era5_downscaled/era5_val.nc')
era5_test.to_netcdf('/content/drive/MyDrive/era5_downscaled/era5_test.nc')

In [4]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nvnsudharsan (nvnsudharsan-the-university-of-texas-at-austin) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
!git clone https://github.com/manmeet3591/xdownscale.git
#!cd /content/xdownscale
!pip install /content/xdownscale/.

Cloning into 'xdownscale'...
remote: Enumerating objects: 456, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 456 (delta 50), reused 8 (delta 1), pack-reused 358 (from 3)
Receiving objects: 100% (456/456), 7.58 MiB | 11.44 MiB/s, done.
Resolving deltas: 100% (248/248), done.
Processing ./xdownscale
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import xarray as xr

def center_crop_2d(da, size=(192, 192)):
    """Crops the DataArray along x and y to the center (192x192)."""
    y, x = da.sizes['y'], da.sizes['x']
    dy, dx = size

    y_start = (y - dy) // 2
    x_start = (x - dx) // 2

    return da.isel(y=slice(y_start, y_start + dy), x=slice(x_start, x_start + dx))

def ensure_time_first(da):
    """Ensures dimensions are ordered (time, y, x)."""
    if 'time' in da.dims:
        return da.transpose('time', 'y', 'x')
    else:
        da = da.expand_dims(dim='time')  # just in case time was squeezed out
        return da.transpose('time', 'y', 'x')

In [7]:
data_dir = "/content/drive/MyDrive/era5_downscaled"

input_da = xr.open_mfdataset([f"{data_dir}/era5_train.nc", f"{data_dir}/era5_val.nc"]).to_array().squeeze()
target_da = xr.open_mfdataset([f"{data_dir}/prism_train.nc", f"{data_dir}/prism_val.nc"]).to_array().squeeze()
input_test = xr.open_dataset(f"{data_dir}/era5_test.nc").to_array().squeeze()
target_test = xr.open_dataset(f"{data_dir}/prism_test.nc").to_array().squeeze()

# Crop spatial dims to 192x192
input_da = center_crop_2d(input_da)
target_da = center_crop_2d(target_da)
input_test = center_crop_2d(input_test)
target_test = center_crop_2d(target_test)

# Ensure all are (time, y, x)
input_da = ensure_time_first(input_da)
target_da = ensure_time_first(target_da)
input_test = ensure_time_first(input_test)
target_test = ensure_time_first(target_test)

# Final check
print("Input:", input_da.shape)
print("Target:", target_da.shape)
print("Input Test:", input_test.shape)
print("Target Test:", target_test.shape)

Input: (2922, 192, 192)
Target: (2922, 192, 192)
Input Test: (730, 192, 192)
Target Test: (730, 192, 192)


In [8]:
import xarray as xr
import numpy as np
import os
import torch
import wandb
from xdownscale import Downscaler  # make sure `Downscaler` is correctly imported from your package

# ---------- CONFIGURATION ----------
model_list = [
    "srcnn", "fsrcnn", "lapsr", "carnm", "falsra", "falsrb", "srresnet",
    "carn", "oisrrk2", "mdsr", "san", "rcan", "unet", "dlgsanet", "dpmn",
    "safmn", "dpt", "distgssr", "swin"
]

data_dir = '/content/drive/MyDrive/era5_downscaled'
output_dir = os.path.join(data_dir, 'model_weights')
os.makedirs(output_dir, exist_ok=True)

# ---------- HELPER: STACK TIME TO SAMPLE ----------
def preprocess_time_as_batch(da):
    return da.stack(sample=("time",)).transpose("sample", "y", "x")

# ---------- STACK FOR TRAINING ----------
input_stack = preprocess_time_as_batch(input_da)
target_stack = preprocess_time_as_batch(target_da)


/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [8]:
input_stack

<xarray.DataArray (sample: 2922, y: 192, x: 192)> Size: 431MB
dask.array<transpose, shape=(2922, 192, 192), dtype=float32, chunksize=(2337, 192, 192), chunktype=numpy.ndarray>
Coordinates:
  * x         (x) float64 2kB -97.94 -97.93 -97.92 ... -96.58 -96.57 -96.57
  * y         (y) float64 2kB 30.94 30.93 30.93 30.92 ... 29.58 29.57 29.57
    variable  <U9 36B 'tp_mm_day'
  * sample    (sample) object 23kB MultiIndex
  * time      (sample) datetime64[ns] 23kB 2015-01-01 2015-01-02 ... 2022-12-31

In [ ]:
# ---------- LOOP THROUGH MODELS ----------
for model_name in model_list:
    run_name = f"{model_name}_run"
    wandb.init(project="xdownscale_Austin_06_15", name=run_name, reinit=True)

    print(f"\n Training model: {model_name}")
    ds = Downscaler(
        input_da=input_stack,
        target_da=target_stack,
        model_name=model_name,
        patch_size=64,
        batch_size=16,
        epochs=100,
        val_split=0.2,
        test_split=0.0,
        device='cuda',
        use_wandb=True,
        patience=15
    )

    # Save best model
    torch.save(ds.model.state_dict(), os.path.join(output_dir, f"{model_name}_best.pth"))

    # ---------- PREDICT DAY-BY-DAY ----------
    preds = []
    for i in range(input_test.time.size):
        input_slice = input_test.isel(time=i)
        pred_slice = ds.predict(input_slice)
        pred_slice = pred_slice.expand_dims(time=[input_test.time[i].values])
        preds.append(pred_slice)

    pred_all = xr.concat(preds, dim="time")
    pred_all.name = "tp_mm_day_pred"
    pred_all = pred_all.assign_coords(x=input_test.x, y=input_test.y)

    # ---------- SAVE OUTPUT ----------
    pred_all.to_netcdf(os.path.join(data_dir, f"pred_{model_name}_test.nc"))

    # ---------- EVALUATE ----------
    true = target_test.values
    predicted = pred_all.values

    rmse = np.sqrt(np.mean((predicted - true) ** 2))
    mae = np.mean(np.abs(predicted - true))
    bias = np.mean(predicted - true)
    corr = np.corrcoef(predicted.ravel(), true.ravel())[0, 1]

    wandb.log({
        "test_mean": float(pred_all.mean().values),
        "test_std": float(pred_all.std().values),
        "test_rmse": rmse,
        "test_mae": mae,
        "test_bias": bias,
        "test_corr": corr
    })

    print(f"Done with {model_name} | RMSE={rmse:.3f}, CORR={corr:.3f}")

    wandb.finish()


 Training model: srcnn
[0] Train: 0.0007 | Val: 0.0006
[10] Train: 0.0007 | Val: 0.0006
Early stopping at epoch 15 with best val_loss: 0.0006
Done with srcnn | RMSE=7.458, CORR=0.462


epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
test_bias,▁
test_corr,▁
test_mae,▁
test_mean,▁
test_rmse,▁
test_std,▁
train_loss,█▅▂▅▄▄▅▃▄▃▃▃▂▁▁▁
val_loss,▄▃▇▂▂▃▂▂▄▅█▅▂▃▅▁
epoch,15
test_bias,1.10021



 Training model: fsrcnn
[0] Train: 0.0016 | Val: 0.0007
[10] Train: 0.0007 | Val: 0.0007
Early stopping at epoch 15 with best val_loss: 0.0007
Done with fsrcnn | RMSE=7.631, CORR=0.434


epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
test_bias,▁
test_corr,▁
test_mae,▁
test_mean,▁
test_rmse,▁
test_std,▁
train_loss,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁
val_loss,▂█▃▂▄▁▂▁▃▁▂▄▁▁▂▁
epoch,15
test_bias,-0.29295



 Training model: lapsr
[0] Train: 0.0007 | Val: 0.0008
[10] Train: 0.0007 | Val: 0.0007
Early stopping at epoch 15 with best val_loss: 0.0008
Done with lapsr | RMSE=8.262, CORR=0.449


...


 Training model: carn
[0] Train: 0.0012 | Val: 0.0007
[10] Train: 0.0007 | Val: 0.0006
[20] Train: 0.0007 | Val: 0.0006
Early stopping at epoch 27 with best val_loss: 0.0006
Done with carn | RMSE=7.329, CORR=0.471


epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇██
test_bias,▁
test_corr,▁
test_mae,▁
test_mean,▁
test_rmse,▁
test_std,▁
train_loss,█▂▂▂▅▁▁▁▁▁▁▁▅▁▁▁▁▂▁▁▁▅▁▁▁▁▁▁
val_loss,▆▆▃█▅▂▁▃▃▅▂▂▁▂▂▁▂▁▁▂▁▁▃▁▂▂▁▂
epoch,27
test_bias,0.18116



 Training model: oisrrk2
[0] Train: 0.0010 | Val: 0.0008
[10] Train: 0.0007 | Val: 0.0007
Early stopping at epoch 18 with best val_loss: 0.0007
Done with oisrrk2 | RMSE=7.404, CORR=0.459


epoch,▁▁▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇██
test_bias,▁
test_corr,▁
test_mae,▁
test_mean,▁
test_rmse,▁
test_std,▁
train_loss,█▄▂▂▂▁▂▁▂▁▁▁▁▁▁▃▁▁▁
val_loss,▅▆▆▁▅▃▄█▃▃▂▂▂▅▅▂▄▂▃
epoch,18
test_bias,-0.43098



 Training model: mdsr
[0] Train: 0.1745 | Val: 0.0026
